In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glioblastoma"
cohort = "GSE249289"

# Input paths
in_trait_dir = "../../input/GEO/Glioblastoma"
in_cohort_dir = "../../input/GEO/Glioblastoma/GSE249289"

# Output paths
out_data_file = "../../output/preprocess/Glioblastoma/GSE249289.csv"
out_gene_data_file = "../../output/preprocess/Glioblastoma/gene_data/GSE249289.csv"
out_clinical_data_file = "../../output/preprocess/Glioblastoma/clinical_data/GSE249289.csv"
json_path = "../../output/preprocess/Glioblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiles of glioblastoma tumorspheres cultured in diverse platforms"
!Series_summary	"We studied five patients with IDH1 wild-type glioblastoma who were newly diagnosed with no treatment history via surgery, chemotherapy, or radiotherapy. Patient-derived glioblastoma tumorspheres (TSs) were established from fresh tissue specimens, and they were cultured in divserse platforms."
!Series_overall_design	"Gene expression profiles of five glioblastoma tumorspheres cultured in diverse platforms (collagen, normal ECM, tumor ECM, and mouse xenograft)"
Sample Characteristics Dictionary:
{0: ['tissue: Brain'], 1: ['Sex: Male', 'Sex: Female'], 2: ['age: 61', 'age: 56', 'age: 57', 'age: 67'], 3: ['tumorsphere: TS13-20', 'tumorsphere: TS13-64', 'tumorsphere: TS14-08', 'tumorsphere: TS14-15', 'tumorsphere: TS15-88'], 4: ['culture platform: Collagen', 'culture platform: nECM', 'culture platform: tECM', 'culture platform: mouse xenograft']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Let's start by analyzing the data availability

# 1. Gene Expression Data Availability
# Based on the Series title and summary, this dataset appears to contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the Sample Characteristics Dictionary:

# 2.1 Trait (Glioblastoma)
# Since all subjects have glioblastoma, we can use tumorsphere ID at index 3 as our trait variable
trait_row = 3  # tumorsphere ID

def convert_trait(value):
    if not value or ':' not in value:
        return None
    # Extract tumorsphere ID
    ts_id = value.split(":", 1)[1].strip()
    # For simplicity in this study, return 1 since all samples are glioblastoma
    return 1

# 2.2 Age information is at index 2
age_row = 2

def convert_age(value):
    if not value or ':' not in value:
        return None
    # Extract age value after colon
    age_str = value.split(":", 1)[1].strip()
    try:
        # Convert to integer (continuous value)
        return int(age_str)
    except ValueError:
        return None

# 2.3 Gender information is at index 1
gender_row = 1

def convert_gender(value):
    if not value or ':' not in value:
        return None
    # Extract gender value after colon
    gender = value.split(":", 1)[1].strip().lower()
    # Convert to binary: 0 for female, 1 for male
    if gender == 'female':
        return 0
    elif gender == 'male':
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step as we don't have access to the clinical_data from a previous step
# This would need to be completed when the clinical_data is available


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 62
Header line: "ID_REF"	"GSM7933102"	"GSM7933103"	"GSM7933104"	"GSM7933105"	"GSM7933106"	"GSM7933107"	"GSM7933108"	"GSM7933109"	"GSM7933110"	"GSM7933111"	"GSM7933112"	"GSM7933113"	"GSM7933114"	"GSM7933115"	"GSM7933116"	"GSM7933117"	"GSM7933118"	"GSM7933119"	"GSM7933120"	"GSM7933121"	"GSM7933122"	"GSM7933123"	"GSM7933124"	"GSM7933125"	"GSM7933126"	"GSM7933127"	"GSM7933128"	"GSM7933129"	"GSM7933130"	"GSM7933131"	"GSM7933132"	"GSM7933133"	"GSM7933134"	"GSM7933135"	"GSM7933136"	"GSM7933137"	"GSM7933138"	"GSM7933139"	"GSM7933140"	"GSM7933141"	"GSM7933142"	"GSM7933143"	"GSM7933144"	"GSM7933145"	"GSM7933146"	"GSM7933147"	"GSM7933148"
First data line: "ILMN_1343291"	36765.52574	36765.52574	34572.79785	35950.13809	36765.52574	37899.98532	39162.20468	37232.73	37899.98532	39162.20468	39162.20468	39162.20468	39162.20468	39162.20468	37232.73	39162.20468	36356.66553	39162.20468	30675.89726	27449.77672	10234.08017	28621.69313	37232.73	39162.20468	34940.55083	35950.13809	363

Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers start with "ILMN_" which indicates they are Illumina probe IDs,
# not human gene symbols. Illumina IDs need to be mapped to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE249289
Line 6: !Series_title = Gene expression profiles of glioblastoma tumorspheres cultured in diverse platforms
Line 7: !Series_geo_accession = GSE249289
Line 8: !Series_status = Public on Dec 09 2023
Line 9: !Series_submission_date = Dec 04 2023
Line 10: !Series_last_update_date = Dec 09 2023
Line 11: !Series_summary = We studied five patients with IDH1 wild-type glioblastoma who were newly diagnosed with no treatment history via surgery, chemotherapy, or radiotherapy. Patient-derived glioblastoma tumorspheres (TSs) were established from fresh tissue specimens, and they were cultured in divserse platforms.
Line 12: !Series_overall_design = Gene expression profiles of five glioblastoma tumorspheres cu


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# 1. First, determine the mapping columns
# From the previous outputs, we can see:
# - In gene_data, gene IDs are in the format "ILMN_XXXXXXX"
# - In gene_annotation, these IDs are in the "ID" column
# - The gene symbols are in the "Symbol" column

# 2. Create the gene mapping dataframe
# Extract the relevant columns (ID and Symbol) for mapping
mapping_df = gene_annotation[['ID', 'Symbol']].copy()

# Filter out entries with empty gene symbols and convert to string type
mapping_df = mapping_df.dropna(subset=['Symbol'])
mapping_df = mapping_df.astype({'ID': 'str', 'Symbol': 'str'})

# Rename Symbol column to Gene to match the expected structure in apply_gene_mapping function
mapping_df = mapping_df.rename(columns={'Symbol': 'Gene'})

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
# Use the library function to handle the many-to-many relationship between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Verify the result by checking the head of the gene_data dataframe
print("Converted gene expression data preview:")
print(gene_data.head())
print(f"Shape of gene expression data: {gene_data.shape}")

# Save the gene expression data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Converted gene expression data preview:
       GSM7933102  GSM7933103  GSM7933104  GSM7933105  GSM7933106  GSM7933107  \
Gene                                                                            
A1BG     51.16928    66.11620    35.05003    25.08680    17.77941    34.71231   
A1CF     -5.32805    -6.33568     9.02714    31.75687    -5.53339     5.60693   
A26C3    -9.95525    -7.17041   -15.59057    -0.76042    32.17539   -13.53330   
A2BP1   -10.09491   -32.64002   -22.01939    -9.12635   -39.38100   -38.72339   
A2LD1    26.42643    18.98107    25.65493     8.60538    14.89496    -4.90494   

       GSM7933108  GSM7933109  GSM7933110  GSM7933111  ...  GSM7933139  \
Gene                                                   ...               
A1BG     36.23326    45.66879    69.21304    32.32465  ...  2221.74373   
A1CF      2.77058    17.76274    -2.80695     3.50213  ...  6571.60134   
A26C3     1.43785    -0.70152   -15.03475   -11.21337  ...   261.59080   
A2BP1   -17.16718   -2

Gene expression data saved to ../../output/preprocess/Glioblastoma/gene_data/GSE249289.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the gene expression data saved in step 6
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print(f"Loaded gene data shape: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data (overwrite the previous file with normalized data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Generate and save clinical data if it doesn't exist
if not os.path.exists(out_clinical_data_file):
    print("Clinical data file not found, generating it now")
    # Get the SOFT and matrix files again
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Get the clinical data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Define conversion functions based on the sample characteristics from step 1
    def convert_trait(value):
        if not value or ':' not in value:
            return None
        # Extract tumorsphere ID
        ts_id = value.split(":", 1)[1].strip()
        # For simplicity in this study, return 1 since all samples are glioblastoma
        return 1

    def convert_age(value):
        if not value or ':' not in value:
            return None
        # Extract age value after colon
        age_str = value.split(":", 1)[1].strip()
        try:
            # Convert to integer (continuous value)
            return int(age_str)
        except ValueError:
            return None

    def convert_gender(value):
        if not value or ':' not in value:
            return None
        # Extract gender value after colon
        gender = value.split(":", 1)[1].strip().lower()
        # Convert to binary: 0 for female, 1 for male
        if gender == 'female':
            return 0
        elif gender == 'male':
            return 1
        else:
            return None
            
    # Define row indices based on sample characteristics from step 1
    trait_row = 3  # tumorsphere ID
    age_row = 2
    gender_row = 1
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    clinical_df = clinical_features
else:
    clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)

print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(clinical_df.head())

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
if 'Age' in linked_data.columns:
    print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
if 'Gender' in linked_data.columns:
    print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")

gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
if gene_cols:
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")

cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
is_trait_biased = False
if len(cleaned_data) > 0:
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
    is_trait_biased = trait_biased
else:
    print("No data remains after handling missing values.")
    is_trait_biased = True

# 6. Final validation and save
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=len(normalized_gene_data) > 0, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note=f"Dataset contains gene expression data for {trait} analysis."
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Loaded gene data shape: (21464, 47)
Gene data shape after normalization: (20259, 47)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Normalized gene data saved to ../../output/preprocess/Glioblastoma/gene_data/GSE249289.csv
Clinical data file not found, generating it now
Clinical data shape: (3, 47)
Clinical data preview:
              GSM7933102  GSM7933103  GSM7933104  GSM7933105  GSM7933106  \
Glioblastoma         1.0         1.0         1.0         1.0         1.0   
Age                 61.0        61.0        61.0        61.0        61.0   
Gender               1.0         1.0         1.0         1.0         1.0   

              GSM7933107  GSM7933108  GSM7933109  GSM7933110  GSM7933111  ...  \
Glioblastoma         1.0         1.0         1.0         1.0         1.0  ...   
Age                 61.0        61.0        61.0        61.0        61.0  ...   
Gender               1.0         1.0         1.0         1.0         1.0  ...   

              GSM7933139  GSM7933140  GSM7933141  GSM7933142  GSM7933143  \
Glioblastoma         1.0         1.0         1.0         1.0         1.0   
Age                 67.0   

Data shape after handling missing values: (47, 20262)
Quartiles for 'Glioblastoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Glioblastoma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 57.0
  50% (Median): 61.0
  75%: 61.0
Min: 56.0
Max: 67.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 13 occurrences. This represents 27.66% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data was determined to be unusable or empty and was not saved
